In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Loading Data

In [0]:
'''Loading data from 2005-2015'''

df=[] #len=11, each = 1yr
for i in range(2005,2016):
    url = '/content/drive/My Drive/Colab Notebooks/Fondation Final Project_Mortality/data/mortality/'+str(i)+'_data.csv'
    df1 = pd.read_csv(url)
    df.append(df1)

del df1

FileNotFoundError: ignored

# Prepossessing Data

In [0]:
'''
Create new column - '5y_age_band_recode'
'''
#create a dictionary to map age_recode_52 to 5 yr age band
dic = dict()
for i in range(1,53):
    if i<=26:
        dic[i] = 1
    else:
        dic[i] = i-26+1
#print(dic)
for i in range(11):
    df[i]['5y_age_band_recode'] = df[i]['age_recode_52'].map(dic)
df[0][:2]

,resident_status,education_1989_revision,education_2003_revision,education_reporting_flag,month_of_death,sex,detail_age_type,detail_age,age_substitution_flag,age_recode_52,...,record_condition_19,record_condition_20,race,bridged_race_flag,race_imputation_flag,race_recode_3,race_recode_5,hispanic_origin,hispanic_originrace_recode,5y_age_band_recode
0,1,11.0,NaN,0,1,F,1,45,NaN,35,...,NaN,NaN,1,NaN,NaN,1,1,100,6,10
1,1,13.0,NaN,0,1,M,1,61,NaN,38,...,NaN,NaN,1,NaN,NaN,1,1,100,6,13


In [0]:
'''
Recode the 39_cause_recode to rankable causes
'''

'''
#create column - 50_cause_recode
#rankable death causes

causepd = pd.read_csv('data/50Causes.csv')
#causedic = causepd.set_index('113_cause_recode').T.to_dict('list')
causedic = causepd.set_index('113_cause_recode').to_dict()['50_cause_recode']
print(causedic)

for i in range(11):
    df[i]['50_cause_recode'] = df[i]['113_cause_recode'].map(causedic)
    df[i][['50_cause_recode']] = df[i][['50_cause_recode']].astype(int)
del causedic, causepd'''


recodedf = pd.read_csv('data/map data/rankable_cause_recode_map.csv')

recodedict = recodedf.set_index('39_cause_recode').to_dict()['27_rankable_cause']

for i in range(11):
    df[i]['rankable_cause_recode'] = df[i]['39_cause_recode'].map(recodedict)
del recodedf,recodedict

df[0][:1]

,resident_status,education_1989_revision,education_2003_revision,education_reporting_flag,month_of_death,sex,detail_age_type,detail_age,age_substitution_flag,age_recode_52,...,record_condition_20,race,bridged_race_flag,race_imputation_flag,race_recode_3,race_recode_5,hispanic_origin,hispanic_originrace_recode,5y_age_band_recode,rankable_cause_recode
0,1,11.0,NaN,0,1,F,1,45,NaN,35,...,NaN,1,NaN,NaN,1,1,100,6,10,4


In [0]:
'''
drop unused columns
reduce memory pressure
'''
for i in range(11):
    #determine the columns to drop
    all_col = df[i].columns.values
    used_col = ['education_1989_revision','education_2003_revision', 'education_reporting_flag',
             'month_of_death', 'sex', 'marital_status', 'day_of_week_of_death',
             'current_data_year', 'injury_at_work', 'manner_of_death', 'activity_code', 'race',
                'detail_age', '5y_age_band_recode', 'rankable_cause_recode']
    unused_col = list(set(all_col).difference(set(used_col)))
    
    df[i] = df[i].drop(unused_col,axis=1)
    print('finish droping unused columns for year', i+2005)
    print('column number after drop:',len(df[i].columns.values))
del all_col, unused_col

finish droping unused columns for year 2005
column number after drop: 15
finish droping unused columns for year 2006
column number after drop: 15
finish droping unused columns for year 2007
column number after drop: 15
finish droping unused columns for year 2008
column number after drop: 15
finish droping unused columns for year 2009
column number after drop: 15
finish droping unused columns for year 2010
column number after drop: 15
finish droping unused columns for year 2011
column number after drop: 15
finish droping unused columns for year 2012
column number after drop: 15
finish droping unused columns for year 2013
column number after drop: 15
finish droping unused columns for year 2014
column number after drop: 15
finish droping unused columns for year 2015
column number after drop: 15


In [0]:
'''
Process NaN value
'''
#print out missing value situation
for i in range(11):
    print('Year',2005+i, 'missing value\n',df[i].isnull().sum(), sep='')
    print('------')

Year2005missing value
education_1989_revision      965336
education_2003_revision     1487170
education_reporting_flag          0
month_of_death                    0
sex                               0
detail_age                        0
marital_status                    0
day_of_week_of_death              0
current_data_year                 0
injury_at_work                    0
manner_of_death              732891
activity_code               2281702
race                              0
5y_age_band_recode                0
rankable_cause_recode             0
dtype: int64
------
Year2006missing value
education_1989_revision     1174880
education_2003_revision     1255845
education_reporting_flag          0
month_of_death                    0
sex                               0
detail_age                        0
marital_status                    0
day_of_week_of_death              0
current_data_year                 0
injury_at_work                    0
manner_of_death              693733


In [0]:
for i in range(11):
    url = 'data/mortality_selected/'+str(2005+i)+'.csv'
    print('Writing:', url)
    df[i].to_csv(url, index=False, sep=',')
print('Finish!')

Writing: data/mortality_selected/2005.csv
Writing: data/mortality_selected/2006.csv
Writing: data/mortality_selected/2007.csv
Writing: data/mortality_selected/2008.csv
Writing: data/mortality_selected/2009.csv
Writing: data/mortality_selected/2010.csv
Writing: data/mortality_selected/2011.csv
Writing: data/mortality_selected/2012.csv
Writing: data/mortality_selected/2013.csv
Writing: data/mortality_selected/2014.csv
Writing: data/mortality_selected/2015.csv
Finish!
